In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:

from pathlib import Path

import flwr
import torch
from flwr.simulation import run_simulation
from torch.optim.lr_scheduler import CosineAnnealingLR

from fl_g13.architectures import BaseDino
from fl_g13.fl_pytorch.client_app import get_client_app
from fl_g13.fl_pytorch.server_app import get_server_app

2025-06-03 22:50:19.178 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13


In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
# disable_progress_bar()

Training on cuda
Flower 1.17.0 / PyTorch 2.6.0+cu118


# Login wandb

In [5]:
!pip install wandb


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
## read .env file
import dotenv

dotenv.load_dotenv()


True

In [7]:
import wandb

# login by key in .env file
# WANDB_API_KEY = dotenv.dotenv_values()["WANDB_API_KEY"]
WANDB_API_KEY = 'd8a0d7bc0ada694ba9c7f26bd159620f0326a74f'
wandb.login(key=WANDB_API_KEY)

C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\.venv\lib\site-packages\notebook\utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\ADMIN\_netrc
wandb: Currently logged in as: thanhnv-it23 (stefano-gamba-social-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Build module local

Build module local such that ClientApp can use it

In [8]:
!pip install -e ..

Obtaining file:///C:/Users/ADMIN/Desktop/BACKUP/study/Italy/polito/classes/20242/deep%20learning/project/source_code/fl-g13

  Installing build dependencies: started

  Installing build dependencies: finished with status 'done'

[notice] A new release of pip is available: 25.0.1 -> 25.1.1


[notice] To update, run: python.exe -m pip install --upgrade pip

  Checking if build backend supports build_editable: started


  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'


### Download missing module for clients

Dino model,that is serialized and sent to client by server, require some modules that have to download from source code of dino model


In [9]:

import urllib.request


def download_if_not_exists(file_path: str, file_url: str):
    """
    Checks if a file exists at the given path. If it does not, downloads it from the specified URL.

    Parameters:
    - file_path (str): The local path to check and save the file.
    - file_url (str): The URL from which to download the file.
    """
    if not os.path.exists(file_path):
        print(f"'{file_path}' not found. Downloading from {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
            print("Download complete.")
        except Exception as e:
            print(f"Failed to download file: {e}")
    else:
        print(f"'{file_path}' already exists.")

In [10]:
download_if_not_exists("vision_transformer.py",
                       "https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/vision_transformer.py")
download_if_not_exists("utils.py",
                       "https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/utils.py")


'vision_transformer.py' already exists.
'utils.py' already exists.


# FL

## Configs

In [11]:
DEBUG = True

In [12]:
K = 100
C = 0.1
J = 4
# partition_type = 'iid'
partition_type = 'shard'
Nc = 10

## only for partition_type = 'shard'
num_shards_per_partition = Nc

# checkpoint_dir = "/content/drive/MyDrive/mldl_fl/fl_dino/non_iid/checkpoints"
# ##
# os.makedirs(checkpoint_dir, exist_ok=True)


# Model Hyper-parameters
head_layers = 3
head_hidden_size = 512
dropout_rate = 0.0
unfreeze_blocks = 0

# Training Hyper-parameters
batch_size = 64
lr = 1e-3
momentum = 0.9
weight_decay = 1e-5
T_max = 8
eta_min = 1e-5

save_every = 5
fraction_fit = C  # Sample of available clients for training
fraction_evaluate = 0.1  # Sample 50% of available clients for evaluation
min_fit_clients = 10  # Never sample less than 10 clients for training
min_evaluate_clients = 5  # Never sample less than 5 clients for evaluation
min_available_clients = 10  # Wait until all 10 clients are available
device = DEVICE
## Wandb config
use_wandb = True
wandb_config = {
    # wandb param
    'name': 'FL_Dino_Baseline_iid',
    'project_name': "FL_Dino_CIFAR100_experiment",
    # model config param
    "fraction_fit": fraction_fit,
    "lr": lr,
    "momentum": momentum,
    'partition_type': partition_type,
    'K': K,
    'C': C,
    'J': J,
}



### Download missing module for clients

Dino model,that is serialized and sent to client by server, require some modules that have to download from source code of dino model

In [13]:
import os
import urllib.request


def download_if_not_exists(file_path: str, file_url: str):
    """
    Checks if a file exists at the given path. If it does not, downloads it from the specified URL.

    Parameters:
    - file_path (str): The local path to check and save the file.
    - file_url (str): The URL from which to download the file.
    """
    if not os.path.exists(file_path):
        print(f"'{file_path}' not found. Downloading from {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
            print("Download complete.")
        except Exception as e:
            print(f"Failed to download file: {e}")
    else:
        print(f"'{file_path}' already exists.")


download_if_not_exists("vision_transformer.py",
                       "https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/vision_transformer.py")
download_if_not_exists("utils.py",
                       "https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/utils.py")


'vision_transformer.py' already exists.
'utils.py' already exists.


## Define model , optimizer and loss function

## Run the training


In [14]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda":
    backend_config["client_resources"] = {"num_cpus": 1, "num_gpus": 1}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

In [ ]:
from fl_g13.fl_pytorch.datasets import reset_partition
from fl_g13.modeling import load_or_create
from fl_g13.editing import SparseSGDM
from torch.optim import SGD

K = 100
C = 0.1

num_rounds = 200
partition_type = 'shard'
evaluate_each = 1
# partition_type = 'iid'
NUM_CLIENTS = K
Js = [8]
Ncs = [1, 5, 10, 50]

## only for partition_type = 'shard'

## Wandb config
use_wandb = True

project_name = "FL_Dino_CIFAR100_baseline_v4"

current_path = Path.cwd()
model_save_path = current_path / f"../models/fl_dino_v4/non_iid"

model_config={
    "head_layers": 3,
    "head_hidden_size": 512,
    "dropout_rate": 0.0,
    "unfreeze_blocks": 0,
}

for Nc in Ncs:
    reset_partition()
    for J in Js:
        print('-' * 200)
        print(f"Training Non IId model")
        print(f"Nc: {Nc}, J: {J}")
        checkpoint_dir = f"{model_save_path}/{Nc}_{J}/editing"
        print(f'checkpoint_dir:{checkpoint_dir}')
        # Model
        model, start_epoch = load_or_create(
            path=checkpoint_dir,
            model_class=BaseDino,
            model_config=model_config,
            optimizer=None,
            scheduler=None,
            device=device,
            verbose=True,
        )

        model.to(DEVICE)

        optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
        criterion = torch.nn.CrossEntropyLoss()
        scheduler = CosineAnnealingLR(
            optimizer=optimizer,
            T_max=T_max,
            eta_min=eta_min
        )

        ## unfreeze blocks
        num_blocks = 0
        model.unfreeze_blocks(num_blocks)
        num_shards_per_partition = Nc

        ##
        os.makedirs(checkpoint_dir, exist_ok=True)

        name = f"FL_Dino_Baseline_model_non_iid_{Nc}_{J}"
        
        wandb_config = {
            # wandb param
            'name': name,
            'project_name': project_name,
            'run_id': f"{name}",
            # model config param
            "fraction_fit": fraction_fit,
            "lr": lr,
            "momentum": momentum,
            'partition_type': partition_type,
            'K': K,
            'C': C,
            'J': J,
            'Nc': Nc
        }
        # model editing config
        model_editing = False
        mask_type = 'global'
        sparsity = 0.8
        mask = None
        model_editing_batch_size = 16
        if model_editing:
            # Create a dummy mask for SparseSGDM
            init_mask = [torch.ones_like(p, device=p.device) for p in
                         model.parameters()]  # Must be done AFTER the model is moved to the device
            # Optimizer, scheduler, and loss function
            optimizer = SparseSGDM(
                model.parameters(),
                mask=init_mask,
                lr=lr,
                momentum=momentum,
                weight_decay=weight_decay
            )

        client = get_client_app(
            model=model,
            optimizer=optimizer,
            criterion=criterion,
            device=DEVICE,
            partition_type=partition_type,
            local_epochs=1,
            local_steps=J,
            batch_size=batch_size,
            num_shards_per_partition=num_shards_per_partition,
            scheduler=scheduler,
            verbose=0,
            model_editing=model_editing,
            mask_type=mask_type,
            sparsity=sparsity,
            mask=mask,
            model_editing_batch_size=model_editing_batch_size,
            mask_func=None
        )
        compute_round = num_rounds + 1 - start_epoch
        server = get_server_app(checkpoint_dir=checkpoint_dir,
                                model_class=model,
                                optimizer=optimizer,
                                criterion=criterion,
                                scheduler=scheduler,
                                num_rounds=compute_round,
                                fraction_fit=fraction_fit,
                                fraction_evaluate=fraction_evaluate,
                                min_fit_clients=min_fit_clients,
                                min_evaluate_clients=min_evaluate_clients,
                                min_available_clients=min_available_clients,
                                device=device,
                                use_wandb=use_wandb,
                                wandb_config=wandb_config,
                                save_every=save_every,
                                prefix='fl_baseline',
                                evaluate_each=evaluate_each
                                )
        # Run simulation
        run_simulation(
            server_app=server,
            client_app=client,
            num_supernodes=NUM_CLIENTS,
            backend_config=backend_config
        )

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Training Non IId model
Nc: 1, J: 8
checkpoint_dir:C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\notebooks\..\models\fl_baseline\noniid/1_8/editing
⚠️ No checkpoint found at C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\notebooks\..\models\fl_baseline\noniid/1_8/editing. Creating a new model.


Using cache found in C:\Users\ADMIN/.cache\torch\hub\facebookresearch_dino_main


⚠️ No checkpoint found at C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\notebooks\..\models\fl_baseline\noniid/1_8/editing. Creating a new model.
[Server] Server on device: cuda:0
[Server] CUDA available in client: True
Using strategy 'CustomFedAvg' (default option)


C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\.venv\lib\site-packages\wandb\analytics\sentry.py:259: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}  # noqa


INFO :      Starting Flower ServerApp, config: num_rounds=200, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


...

Eval progress: 100%|██████████| 313/313 [00:41<00:00,  7.48batch/s]
INFO :      [Round 0] Centralized Evaluation - Loss: 4.7842, Metrics: {'centralized_accuracy': 0.0075}
INFO :      initial parameters (loss, other metrics): 4.784153261885476, {'centralized_accuracy': 0.0075}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
(ClientAppActor pid=8112) 2025-06-03 22:51:41.901 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13


(ClientAppActor pid=8112) [Client] Client on device: cuda:0
(ClientAppActor pid=8112) [Client] CUDA available in client: True
(ClientAppActor pid=8112) No prefix/name for the model was provided, choosen prefix/name: sleepy_arbok_83
(ClientAppActor pid=8112) 
(ClientAppActor pid=8112) Step 1/4 | Total batches: 7
(ClientAppActor pid=8112) Step 2/4 | Total batches: 7
(ClientAppActor pid=8112) Step 3/4 | Total batches: 7
(ClientAppActor pid=8112) Step 4/4 | Total batches: 7


(ClientAppActor pid=8112) C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\.venv\lib\site-packages\torch\optim\lr_scheduler.py:217: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
(ClientAppActor pid=8112)   warnings.warn(


(ClientAppActor pid=8112) 🚀 Epoch 1/8 (12.50%) Completed
(ClientAppActor pid=8112) 	📊 Training Loss: 2.9998
(ClientAppActor pid=8112) 	✅ Training Accuracy: 45.31%
(ClientAppActor pid=8112) 	⏳ Elapsed Time: 2.75s | ETA: 19.22s
(ClientAppActor pid=8112) 	🕒 Completed At: 22:52
(ClientAppActor pid=8112) 
(ClientAppActor pid=8112) Step 1/4 | Total batches: 7
(ClientAppActor pid=8112) Step 2/4 | Total batches: 7
(ClientAppActor pid=8112) Step 3/4 | Total batches: 7
(ClientAppActor pid=8112) Step 4/4 | Total batches: 7
(ClientAppActor pid=8112) 🚀 Epoch 2/8 (25.00%) Completed
(ClientAppActor pid=8112) 	📊 Training Loss: 0.0231
(ClientAppActor pid=8112) 	✅ Training Accuracy: 100.00%
(ClientAppActor pid=8112) 	⏳ Elapsed Time: 1.21s | ETA: 7.28s
(ClientAppActor pid=8112) 	🕒 Completed At: 22:52
(ClientAppActor pid=8112) 
(ClientAppActor pid=8112) Step 1/4 | Total batches: 7
(ClientAppActor pid=8112) Step 2/4 | Total batches: 7
(ClientAppActor pid=8112) Step 3/4 | Total batches: 7
(ClientAppActor pi

ERROR :     An exception was raised when processing a message by RayBackend
ERROR :     The actor 27e1718be09491861a618ae201000000 is unavailable: The actor is temporarily unavailable: RpcError: RPC Error message: Connection reset; RPC Error details: . The task may or maynot have been executed on the actor.
ERROR :     Traceback (most recent call last):
  File "C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\.venv\lib\site-packages\flwr\server\superlink\fleet\vce\vce_api.py", line 111, in worker
    out_mssg, updated_context = backend.process_message(message, context)
  File "C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\.venv\lib\site-packages\flwr\server\superlink\fleet\vce\backend\raybackend.py", line 187, in process_message
    raise ex
  File "C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\.venv\lib\site-packages